In [4]:
import requests
import pandas as pd
import numpy as np
from selenium.common.exceptions import NoSuchElementException
try:
    from selenium.webdriver import Chrome
except:
    !pip install selenium
    from selenium.webdriver import Chrome

In [5]:
def statscraper(driver,link):
    # Abrir web
    driver.get(link)

    # Extraer primera pestaña
    container = driver.find_element_by_class_name('most-played-heroes')
    info = container.find_element_by_tag_name('tbody').text.split('\n')
    npinfo = np.array(info)
    pdinfo = pd.DataFrame(np.split(npinfo,len(info)/5))
    pdinfo.columns = ['hero','date','matches','wr','kda']

    # Extraer segunda pestaña
    container = driver.find_element_by_class_name('most-played-heroes')
    clickme = container.find_element_by_css_selector('[data-group = "2"]')
    clickme.click()
    info2 = container.find_element_by_tag_name('tbody').text.split('\n')
    npinfo2 = np.array(info2)
    pdinfo2 = pd.DataFrame(np.split(npinfo2,len(info2)/5))
    pdinfo2.columns = ['hero','date','kills','deaths','assists']

    # Extraer tercera pestaña
    container = driver.find_element_by_class_name('most-played-heroes')
    clickme = container.find_element_by_css_selector('[data-group = "3"]')
    clickme.click()
    info3 = container.find_element_by_tag_name('tbody').text.split('\n')
    npinfo3 = np.array(info3)
    pdinfo3 = pd.DataFrame(np.split(npinfo3,len(info3)/6))
    pdinfo3.columns = ['hero','date','lh','denies','gpm','xpm']

    # Consolidar y hacer modificaciones
    final = pdinfo.merge(pdinfo2,how='left',on=['hero','date'])
    final = final.merge(pdinfo3,how='left',on=['hero','date'])
    final['wr'] = final['wr'].apply(lambda x : x[:-1])
    cols = final.columns
    final[cols] = final[cols].apply(pd.to_numeric, errors='coerce')
    final['wins'] = final['matches']*final['wr']/100
    final = final.drop(['date','hero','kda','wr'],axis=1)
    final = final[pd.notnull(final['kills'])]
    final = final.agg({'matches':'sum','kills':'mean','deaths':'mean','assists':'mean','lh':'mean','denies':'mean','gpm':'mean','xpm':'mean','wins':'sum',})
    if (final['deaths'] < 1):
        final['deaths'] = 1
    return final

In [6]:
def agregado(final):
    final['kda'] = (final['kills'] + final['assists']) / final['deaths']
    final = final.drop(['kills','deaths','assists'])
    final['winrate'] = final['wins']/final['matches']
    final.drop('wins')
    if final['matches'] < 15:
        print('Tiene juegos, pero no los suficientes')
        raise NoSuchElementException
    return final

In [7]:
def agregado2(final):
    final['kda'] = (final['kills'] + final['assists']) / final['deaths']
    final = final.drop(['kills','deaths','assists'])
    final['winrate'] = final['wins']/final['matches']
    final.drop('wins')
    return final

In [8]:
def consolidado(stats):
    df = pd.DataFrame(stats)
    df = df.agg({'matches':'sum','kills':'mean','deaths':'mean','assists':'mean','lh':'mean','denies':'mean','gpm':'mean','xpm':'mean','wins':'sum',})
    return(df)

In [21]:
idfirstblood = '5738'
idlpg = '10702'
idmlpgs1 = '11316'
idmlpgs2 = '11414'

idslpg = [idlpg,idmlpgs1,idmlpgs2]

In [ ]:
thunder = {'kotaro':'95825708','leostyle':'59463394','frank':'407321629','matthew':'155332459','wu':'186837494'}
thunderid='6382242'
infamous = {'timado':'97658618','roboz':'123953528','sacred':'408729743','sladin':'101779337','papita':'28070572'}
infamousid='2672298'
unknown = {'berna':'173971950','mrjeans':'133167741','greedy':'109455705','wij':'86850560','prada':'100758751','roy':'129856548'}
unknownid = '7121518'
team0900 = {'argius':'116080282','jeimari':'455509466','smash':'118073569','ztok':'100175472','jericho':'99672424'}
team0900id = '6266565'
infyoung = {'pakazs':'339235645','kxy':'197574429','drakeel':'969982039','michael':'248180032','accel':'99796146'}
infyoungid = '5992560'
creamesports = {'darkmago':'911671153','pamplona':'242885483','aretes':'278770268','tanya':'113639294','cucahook':'303673743'}
creamid = '7640799' 
creamid = '7389602' #id de incubus

In [12]:
teamid = egoboysid
team = egoboys
teamname='egoboys'

In [59]:
driver = Chrome('chromedriver.exe')
df = pd.DataFrame()
for player,playerid in team.items():
    try:
        playerbyteam = "https://www.dotabuff.com/esports/players/" + playerid + "/heroes?team_id=" + teamid
        temp = statscraper(driver,playerbyteam)
        print(player,'- estadisticas en el equipo')
        agg = agregado(temp)
    except NoSuchElementException:
        lpgstats = []
        for lpgid in idslpg:
            try:
                playerinlpg = "https://www.dotabuff.com/esports/players/" + playerid + "/heroes?league_id=" + lpgid
                temp = statscraper(driver,playerinlpg)
                print(player,'- estadisticas en LPG')
                lpgstats.append(temp)
            except NoSuchElementException:
                pass
            notenough = False
            if len(lpgstats) == 0:
                notenough = True
            else:
                final = consolidado(lpgstats)
                agg = agregado2(final)
                if agg['matches'] < 15:
                    notenough = True
        if notenough:
            player12months = "https://www.dotabuff.com/esports/players/" + playerid + "/heroes?date=year"
            temp = statscraper(driver,player12months)
            print(player,'- estadisticas en los ultimos 12 meses')
            agg = agregado2(temp)
    agg['player']=player
    df = df.append(agg,ignore_index=True)

df = df[['player','kda','lh','gpm','xpm','winrate']]
df['lh'] = df['lh'].astype(int)
df['kda'] = (df['kda']*100).astype(int).astype(float)/100
df['gpm'] = df['gpm'].astype(int)
df['xpm'] = df['xpm'].astype(int)
df['winrate'] = df['winrate']*100
df['winrate'] = (df['winrate']*100).astype(int).astype(float)/100

df.to_csv('./output/csv/'+teamname+'.csv',sep=';')

nao - estadisticas en el equipo
tao - estadisticas en el equipo
Tiene juegos, pero no los suficientes
tao - estadisticas en LPG
tao - estadisticas en los ultimos 12 meses
benjaz - estadisticas en el equipo
d1smar - estadisticas en el equipo
cara - estadisticas en el equipo
